In [12]:
#使用Pandas 載入boston_data.csv
import pandas as pd
#呼叫Pandas的read_csv()載入boston_data.csv
df = pd.read_csv("./boston_data.csv")
#呼叫head()顯示前五筆資料
print(df.head())
print(df.shape)#顯示共有幾筆紀錄和欄位

      crim    zn  indus  chas    nox     rm   age     dis  rad    tax  \
0  0.15876   0.0  10.81   0.0  0.413  5.961  17.5  5.2873  4.0  305.0   
1  0.10328  25.0   5.13   0.0  0.453  5.927  47.2  6.9320  8.0  284.0   
2  0.34940   0.0   9.90   0.0  0.544  5.972  76.7  3.1025  4.0  304.0   
3  2.73397   0.0  19.58   0.0  0.871  5.597  94.9  1.5257  5.0  403.0   
4  0.04337  21.0   5.64   0.0  0.439  6.115  63.0  6.8147  4.0  243.0   

   ptratio   black  lstat  medv  
0     19.2  376.94   9.88  21.7  
1     19.7  396.90   9.22  19.6  
2     18.4  396.24   9.97  20.3  
3     14.7  351.85  21.45  15.4  
4     16.8  393.97   9.43  20.5  
(404, 14)


In [17]:
#建立回歸分析的神經網路
#使用keras 建構神經網路
#適用K-fold 交叉驗證
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense

In [18]:
#指定亂數種子
np.random.seed(7)

In [19]:
#載入資料集
df = pd.read_csv("./boston_data.csv")
dataset = df.values
np.random.shuffle(dataset)

In [20]:
#使用Pandas載入CSV檔案，使用values屬性轉換成NumPy陣列
#分隔成前13個欄位的特徵資料和第14個欄位的標籤資料
X = dataset[:, 0:13]
Y = dataset[:, 13]
X -= X.mean(axis=0)
X /= X.std(axis=0)
X_train, Y_train = X[:283], Y[:283]  #訓練資料(80%)
X_test, Y_test = X[283:], Y[283:]  #測試資料(20%)

In [21]:
#建立build_model()建立模型
#使用比較深的四層神經網路
def build_model():
    model = Sequential()
    model.add(Dense(32, input_shape=(X_train.shape[1], ), activation="relu")) #No.1 隱藏層 神經元素為32
    model.add(Dense(16, activation="relu")) #No.2 隱藏層 神經元素為16
    model.add(Dense(1))
    model.compile(loss="mse", optimizer="adam", metrics=["mae"]) #compile()使用的損失函數MSE,評估標準metrics是MAE
    #反映預測值與標籤值誤差實際情況
    return model

In [22]:
#K-fold交叉驗證
k=4 
nb_val_samples = len(X_train) // k #每一折的樣本數
nb_epochs = 80 #訓練週期
mse_scores = [] #mse_scores紀錄mse每一次的迴圈評估模型
mae_scores = [] #mae_scores紀錄mae每一次的迴圈評估模型

In [23]:
#for迴圈共執行K=4次
#使用切割運算子取出第K個驗證資料集X_val和Y_val
for i in range(k):
    print("Processing Fold #" +str(i))
    X_val = X_train[i*nb_val_samples: (i+1)*nb_val_samples]
    Y_val = Y_train[i*nb_val_samples: (i+1)*nb_val_samples]
    #使用concatenate()結合剩下的折建立X_train_p和Y_train_p
    X_train_p = np.concatenate(
            [X_train[:i*nb_val_samples],
             X_train[(i+1)*nb_val_samples:]], axis=0)
    Y_train_p = np.concatenate(
            [Y_train[:i*nb_val_samples],
             Y_train[(i+1)*nb_val_samples:]], axis=0)
    #呼叫build_model()建立神經網路模型
    model = build_model()
    model.fit(X_train_p, Y_train_p, epochs=nb_epochs, batch_size=16, verbose=0) #fit()訓練模型
    mse, mae = model.evaluate(X_val, Y_val) #evaluate()評估模型
    mse_scores.append(mse) #mse評估結果儲存
    mae_scores.append(mae) #mae評估結果儲存

Processing Fold #0
3/3 [==============================] - 0s 0s/step - loss: 20.6420 - mae: 2.7007
Processing Fold #1
3/3 [==============================] - 0s 0s/step - loss: 33.4167 - mae: 3.6178
Processing Fold #2
3/3 [==============================] - 0s 0s/step - loss: 15.1179 - mae: 2.8787
Processing Fold #3
3/3 [==============================] - 0s 10ms/step - loss: 9.9584 - mae: 2.5848


In [24]:
#顯示完成4次迴圈後的K-fold交叉驗證的MSE和MAE的平均值
print("MSE_val: ", np.mean(mse_scores))
print("MAE_val: ", np.mean(mae_scores))
mse, mae= model.evaluate(X_test, Y_test)
print("MSE_test: ",mse)
print("MAE_test: ",mae)

MSE_val:  19.783714056015015
MAE_val:  2.9454989433288574
4/4 [==============================] - 0s 4ms/step - loss: 10.6027 - mae: 2.3490
MSE_test:  10.60269546508789
MAE_test:  2.349008560180664
